In [3]:
import pandas as pd
PATH = "COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/"
doc = pd.read_csv(PATH + "04-01-2020.csv", encoding='utf-8-sig')
doc.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key
0,45001.0,Abbeville,South Carolina,US,2020-04-01 21:58:49,34.223334,-82.461707,4,0,0,0,"Abbeville, South Carolina, US"
1,22001.0,Acadia,Louisiana,US,2020-04-01 21:58:49,30.295065,-92.414197,47,1,0,0,"Acadia, Louisiana, US"
2,51001.0,Accomack,Virginia,US,2020-04-01 21:58:49,37.767072,-75.632346,7,0,0,0,"Accomack, Virginia, US"
3,16001.0,Ada,Idaho,US,2020-04-01 21:58:49,43.452658,-116.241552,195,3,0,0,"Ada, Idaho, US"
4,19001.0,Adair,Iowa,US,2020-04-01 21:58:49,41.330756,-94.471059,1,0,0,0,"Adair, Iowa, US"


In [7]:
doc = pd.read_csv(PATH + "03-01-2020.csv", encoding='utf-8-sig')
doc.head()

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Latitude,Longitude
0,Hubei,Mainland China,2020-03-01T10:13:19,66907,2761,31536,30.9756,112.2707
1,NaN,South Korea,2020-03-01T23:43:03,3736,17,30,36.0000,128.0000
2,NaN,Italy,2020-03-01T23:23:02,1694,34,83,43.0000,12.0000
3,Guangdong,Mainland China,2020-03-01T14:13:18,1349,7,1016,23.3417,113.4244
4,Henan,Mainland China,2020-03-01T14:13:18,1272,22,1198,33.8820,113.6140


In [19]:
doc = pd.read_csv(PATH + "01-22-2020.csv", encoding='utf-8-sig')
try:
    doc = doc[['Province_State', 'Country_Region', 'Confirmed']]
except:
    doc = doc[['Province/State', 'Country/Region', 'Confirmed']]
    doc.columns = ['Province_State', 'Country_Region', 'Confirmed']
    
doc = doc.dropna(subset=['Confirmed'])
doc = doc.astype({'Confirmed' : 'int64'})
    
doc.head()

,Province_State,Country_Region,Confirmed
0,Anhui,Mainland China,1
1,Beijing,Mainland China,14
2,Chongqing,Mainland China,6
3,Fujian,Mainland China,1
5,Guangdong,Mainland China,26


In [17]:
import json

with open('COVID-19-master/csse_covid_19_data/country_convert.json', 'r', encoding='utf-8-sig') as json_file:
    json_data = json.load(json_file)
    print (json_data.keys())

dict_keys(['Mainland China', 'Macau', 'South Korea', 'Aruba', ' Azerbaijan', 'Bahamas, The', 'Cape Verde', 'Cayman Islands', 'Channel Islands', 'Curacao', 'Czech Republic', 'East Timor', 'Faroe Islands', 'French Guiana', 'Gambia, The', 'Gibraltar', 'Greenland', 'Guadeloupe', 'Guam', 'Guernsey', 'Hong Kong', 'Hong Kong SAR', 'Iran (Islamic Republic of)', 'Ivory Coast', 'Jersey', 'Macao SAR', 'Martinique', 'Mayotte', 'North Ireland', 'Palestine', 'Puerto Rico', 'Republic of Ireland', 'Republic of Korea', 'Republic of Moldova', 'Republic of the Congo', 'Reunion', 'Russian Federation', 'Saint Barthelemy', 'Saint Martin', 'St. Martin', 'Taipei and environs', 'The Bahamas', 'The Gambia', 'UK', 'Vatican City', 'Viet Nam', 'occupied Palestinian territory', 'Taiwan*', 'Malawi', 'South Sudan', 'Western Sahara', 'Namibia'])


In [14]:
def func(row):
    if row['Country_Region'] in json_data:
        row['Country_Region'] = json_data[row['Country_Region']]
    return row

In [16]:
doc = doc.apply(func, axis=1)
doc.head()

,Province_State,Country_Region,Confirmed
0,Anhui,China,1.0
1,Beijing,China,14.0
2,Chongqing,China,6.0
3,Fujian,China,1.0
4,Gansu,China,NaN


In [34]:
import json

with open('COVID-19-master/csse_covid_19_data/country_convert.json', 'r', encoding='utf-8-sig') as json_file:
    json_data = json.load(json_file)

def country_name_convert(row):
    if row['Country_Region'] in json_data:
        return json_data[row['Country_Region']]
    return row['Country_Region']

def create_dateframe(filename):

    doc = pd.read_csv(PATH + filename, encoding='utf-8-sig')
    try:
        doc = doc[['Country_Region', 'Confirmed']]
    except:
        doc = doc[['Country/Region', 'Confirmed']] 
        doc.columns = ['Country_Region', 'Confirmed']
    doc = doc.dropna(subset=['Confirmed']) 
    doc['Country_Region'] = doc.apply(country_name_convert, axis=1) 
    doc = doc.astype({'Confirmed': 'int64'}) 
    doc = doc.groupby('Country_Region').sum() 

    date_column = filename.split(".")[0].lstrip('0').replace('-', '/') 
    doc.columns = [date_column]
    return doc

In [31]:
doc1 = create_dateframe("01-22-2020.csv")
doc2 = create_dateframe("04-01-2020.csv")

In [37]:
tmp = "03-21-2020.csv"
tmp = tmp.split(".")[0].lstrip('0').replace('-','/')
tmp

'3/21/2020'

In [32]:
country_info = pd.read_csv("COVID-19-master/csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv", encoding='utf-8-sig')

In [30]:
test_df = pd.merge(doc, country_info, how = 'left', on='Country_Region')
test_df.isnull().sum()
nan_rows = test_df[test_df['iso2'].isnull()]
nan_rows.head()

,Province_State_x,Country_Region,Confirmed,UID,iso2,iso3,code3,FIPS,Admin2,Province_State_y,Lat,Long_,Combined_Key,Population
0,Anhui,Mainland China,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Beijing,Mainland China,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Chongqing,Mainland China,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fujian,Mainland China,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Guangdong,Mainland China,26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
doc = doc.apply(func, axis=1)
doc.head()

,Province_State,Country_Region,Confirmed
0,Anhui,China,1
1,Beijing,China,14
2,Chongqing,China,6
3,Fujian,China,1
5,Guangdong,China,26


In [47]:
import json

with open('COVID-19-master/csse_covid_19_data/country_convert.json', 'r', encoding='utf-8-sig') as json_file:
    json_data = json.load(json_file)

def country_name_convert(row):
    if row['Country_Region'] in json_data:
        return json_data[row['Country_Region']]
    return row['Country_Region']

def create_dateframe(filename):

    doc = pd.read_csv(PATH + filename, encoding='utf-8-sig')
    try:
        doc = doc[['Country_Region', 'Confirmed']]
    except:
        doc = doc[['Country/Region', 'Confirmed']] 
        doc.columns = ['Country_Region', 'Confirmed']
    doc = doc.dropna(subset=['Confirmed']) 
    doc['Country_Region'] = doc.apply(country_name_convert, axis=1) 
    doc = doc.astype({'Confirmed': 'int64'}) 
    doc = doc.groupby('Country_Region').sum() 

    date_column = filename.split(".")[0].lstrip('0').replace('-', '/') 
    doc.columns = [date_column]
    return doc

In [48]:
import os

def generate_dateframe_by_path(PATH):
    
    file_list, csv_list = os.listdir(PATH), list()
    first_doc = True
    for file in file_list :
        if file.split(".")[-1] == 'csv':
            csv_list.append(file)
    csv_list.sort()
    
    for file in csv_list:
        doc = create_dateframe(file)
        if first_doc:
            final_doc, first_doc = doc, False
        else:
            final_doc = pd.merge(final_doc, doc, how='outer', left_index=True, right_index=True)

    final_doc = final_doc.fillna(0)
    return final_doc

In [49]:
PATH = 'COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/'
doc = generate_dateframe_by_path(PATH)
doc

,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,1/31/2020,2/01/2020,2/02/2020,2/03/2020,2/04/2020,2/05/2020,2/06/2020,2/07/2020,2/08/2020,2/09/2020,2/10/2020,2/11/2020,2/12/2020,2/13/2020,2/14/2020,2/15/2020,2/16/2020,2/17/2020,2/18/2020,2/19/2020,2/20/2020,2/21/2020,2/22/2020,2/23/2020,2/24/2020,2/25/2020,2/26/2020,2/27/2020,2/28/2020,2/29/2020,3/01/2020,3/02/2020,3/03/2020,3/04/2020,3/05/2020,3/06/2020,3/07/2020,3/08/2020,3/09/2020,3/10/2020,3/11/2020,3/12/2020,3/13/2020,3/14/2020,3/15/2020,3/16/2020,3/17/2020,3/18/2020,3/19/2020,3/20/2020,3/21/2020,...,10/11/2020,10/12/2020,10/13/2020,10/14/2020,10/15/2020,10/16/2020,10/17/2020,10/18/2020,10/19/2020,10/20/2020,10/21/2020,10/22/2020,10/23/2020,10/24/2020,10/25/2020,10/26/2020,10/27/2020,10/28/2020,10/29/2020,10/30/2020,10/31/2020,11/01/2020,11/02/2020,11/03/2020,11/04/2020,11/05/2020,11/06/2020,11/07/2020,11/08/2020,11/09/2020,11/10/2020,11/11/2020,11/12/2020,11/13/2020,11/14/2020,11/15/2020,11/16/2020,11/17/2020,11/18/2020,11/19/2020,11/20/2020,11/21/2020,11/22/2020,11/23/2020,11/24/2020,11/25/2020,11/26/2020,11/27/2020,11/28/2020,11/29/2020,11/30/2020,12/01/2020,12/02/2020,12/03/2020,12/04/2020,12/05/2020,12/06/2020,12/07/2020,12/08/2020,12/09/2020
Country_Region,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,4.0,4.0,4.0,4.0,5.0,7.0,8.0,11.0,12.0,13.0,15.0,16.0,18.0,20.0,24.0,25.0,29.0,30.0,...,39799.0,39870.0,39928.0,39994.0,40026.0,40088.0,40141.0,40200.0,40287.0,40369.0,40510.0,40626.0,40687.0,40768.0,40833.0,40937.0,41032.0,41145.0,41268.0,41334.0,41425.0,41501.0,41633.0,41728.0,41814.0,41935.0,41975.0,42033.0,42159.0,42297.0,42463.0,42609.0,42795.0,42969.0,43035.0,43240.0,43468.0,43681.0,43924.0,44177.0,44363.0,44503.0,44706.0,44988.0,45174.0,45384.0,45600.0,45723.0,45844.0,46116.0,46274.0,46516.0,46718.0,46837.0,46837.0,47072.0,47306.0,47516.0,47716.0,47851.0
Albania,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,10.0,12.0,23.0,33.0,38.0,42.0,51.0,55.0,59.0,64.0,70.0,76.0,...,15399.0,15570.0,15752.0,15955.0,16212.0,16501.0,16774.0,17055.0,17350.0,17651.0,17948.0,18250.0,18556.0,18858.0,19157.0,19445.0,19729.0,20040.0,20315.0,20634.0,20875.0,21202.0,21523.0,21904.0,22300.0,22721.0,23210.0,23705.0,24206.0,24731.0,25294.0,25801.0,26211.0,26701.0,27233.0,27830.0,28432.0,29126.0,29837.0,30623.0,31459.0,32196.0,32761.0,33556.0,34300.0,34944.0,35600.0,36245.0,36790.0,37625.0,38182.0,39014.0,39719.0,40501.0,41302.0,42148.0,42988.0,43683.0,44436.0,45188.0
Algeria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,5.0,12.0,12.0,17.0,17.0,19.0,20.0,20.0,20.0,24.0,26.0,37.0,48.0,54.0,60.0,74.0,87.0,90.0,139.0,...,53072.0,53325.0,53399.0,53584.0,53777.0,53998.0,54203.0,54402.0,54616.0,54829.0,55081.0,55357.0,55630.0,55880.0,56143.0,56419.0,56706.0,57026.0,57332.0,57651.0,57942.0,58272.0,58574.0,58979.0,59527.0,60169.0,60800.0,61381.0,62051.0,62693.0,63446.0,64257.0,65108.0,65975.0,66819.0,67679.0,68589.0,69591.0,70629.0,71652.0,72755.0,73774.0,74862.0,75867.0,77000.0,78025.0,79110.0,80168.0,81212.0,82221.0,83199.0,84152.0,85084.0,85927.0,86730.0,87502.0,88252.0,88825.0,89416.0,90014.0
Andorra,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,39.0,39.0,53.0,75.0,88.0,...,2696.0,2995.0,2995.0,3190.0,3190.0,3377.0,3377.0,3377.0,3623.0,3623.0,3811.0,3811.0,4038.0,4038.0,40

In [50]:
doc.astype('int64')
doc

,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,1/31/2020,2/01/2020,2/02/2020,2/03/2020,2/04/2020,2/05/2020,2/06/2020,2/07/2020,2/08/2020,2/09/2020,2/10/2020,2/11/2020,2/12/2020,2/13/2020,2/14/2020,2/15/2020,2/16/2020,2/17/2020,2/18/2020,2/19/2020,2/20/2020,2/21/2020,2/22/2020,2/23/2020,2/24/2020,2/25/2020,2/26/2020,2/27/2020,2/28/2020,2/29/2020,3/01/2020,3/02/2020,3/03/2020,3/04/2020,3/05/2020,3/06/2020,3/07/2020,3/08/2020,3/09/2020,3/10/2020,3/11/2020,3/12/2020,3/13/2020,3/14/2020,3/15/2020,3/16/2020,3/17/2020,3/18/2020,3/19/2020,3/20/2020,3/21/2020,...,10/11/2020,10/12/2020,10/13/2020,10/14/2020,10/15/2020,10/16/2020,10/17/2020,10/18/2020,10/19/2020,10/20/2020,10/21/2020,10/22/2020,10/23/2020,10/24/2020,10/25/2020,10/26/2020,10/27/2020,10/28/2020,10/29/2020,10/30/2020,10/31/2020,11/01/2020,11/02/2020,11/03/2020,11/04/2020,11/05/2020,11/06/2020,11/07/2020,11/08/2020,11/09/2020,11/10/2020,11/11/2020,11/12/2020,11/13/2020,11/14/2020,11/15/2020,11/16/2020,11/17/2020,11/18/2020,11/19/2020,11/20/2020,11/21/2020,11/22/2020,11/23/2020,11/24/2020,11/25/2020,11/26/2020,11/27/2020,11/28/2020,11/29/2020,11/30/2020,12/01/2020,12/02/2020,12/03/2020,12/04/2020,12/05/2020,12/06/2020,12/07/2020,12/08/2020,12/09/2020
Country_Region,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,4.0,4.0,4.0,4.0,5.0,7.0,8.0,11.0,12.0,13.0,15.0,16.0,18.0,20.0,24.0,25.0,29.0,30.0,...,39799.0,39870.0,39928.0,39994.0,40026.0,40088.0,40141.0,40200.0,40287.0,40369.0,40510.0,40626.0,40687.0,40768.0,40833.0,40937.0,41032.0,41145.0,41268.0,41334.0,41425.0,41501.0,41633.0,41728.0,41814.0,41935.0,41975.0,42033.0,42159.0,42297.0,42463.0,42609.0,42795.0,42969.0,43035.0,43240.0,43468.0,43681.0,43924.0,44177.0,44363.0,44503.0,44706.0,44988.0,45174.0,45384.0,45600.0,45723.0,45844.0,46116.0,46274.0,46516.0,46718.0,46837.0,46837.0,47072.0,47306.0,47516.0,47716.0,47851.0
Albania,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,10.0,12.0,23.0,33.0,38.0,42.0,51.0,55.0,59.0,64.0,70.0,76.0,...,15399.0,15570.0,15752.0,15955.0,16212.0,16501.0,16774.0,17055.0,17350.0,17651.0,17948.0,18250.0,18556.0,18858.0,19157.0,19445.0,19729.0,20040.0,20315.0,20634.0,20875.0,21202.0,21523.0,21904.0,22300.0,22721.0,23210.0,23705.0,24206.0,24731.0,25294.0,25801.0,26211.0,26701.0,27233.0,27830.0,28432.0,29126.0,29837.0,30623.0,31459.0,32196.0,32761.0,33556.0,34300.0,34944.0,35600.0,36245.0,36790.0,37625.0,38182.0,39014.0,39719.0,40501.0,41302.0,42148.0,42988.0,43683.0,44436.0,45188.0
Algeria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,5.0,12.0,12.0,17.0,17.0,19.0,20.0,20.0,20.0,24.0,26.0,37.0,48.0,54.0,60.0,74.0,87.0,90.0,139.0,...,53072.0,53325.0,53399.0,53584.0,53777.0,53998.0,54203.0,54402.0,54616.0,54829.0,55081.0,55357.0,55630.0,55880.0,56143.0,56419.0,56706.0,57026.0,57332.0,57651.0,57942.0,58272.0,58574.0,58979.0,59527.0,60169.0,60800.0,61381.0,62051.0,62693.0,63446.0,64257.0,65108.0,65975.0,66819.0,67679.0,68589.0,69591.0,70629.0,71652.0,72755.0,73774.0,74862.0,75867.0,77000.0,78025.0,79110.0,80168.0,81212.0,82221.0,83199.0,84152.0,85084.0,85927.0,86730.0,87502.0,88252.0,88825.0,89416.0,90014.0
Andorra,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,39.0,39.0,53.0,75.0,88.0,...,2696.0,2995.0,2995.0,3190.0,3190.0,3377.0,3377.0,3377.0,3623.0,3623.0,3811.0,3811.0,4038.0,4038.0,40

In [51]:
doc.to_csv("COVID-19-master/final_df.csv")